In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import base64
import requests
import os

In [ ]:
#utils
def extract_base64_contents(file):
    with open(file, 'rb') as fp:
        image = base64.b64encode(fp.read()).decode('ascii')
    return image

#expectations
class ServiceRequestError(Exception):
    def __init__(self, status_code, response_msg, payload):
        self.status_code = status_code
        self.response_msg = response_msg
        self.payload = payload

    def __repr__(self):
        print(str(self.response_msg))
        return str(self.response_msg)

    def __str__(self):
        return self.__repr__()

In [ ]:
#consts
dataset_path = "/content/gdrive/MyDrive/MIT_dataset/images/"
file_extension = ".jpg"
subject_id_list = list()
result_map = dict() 

url_base = 'https://api.kairos.com/'
url_enroll = url_base + 'enroll' #stores the faces into a gallery you create
url_list_all_galleries = url_base + 'gallery/list_all' #Lists out all of the galleries you have created.
url_view_gallery = url_base + 'gallery/view' #List images from galerry 
url_view_subject = url_base + 'gallery/view_subject' #List images from galerry  
url_recognize = url_base + 'recognize' #match img against the faces you have already enrolled into a gallery
url_verify = url_base + 'verify' #match img against others in a gallery
url_remove_gallery = url_base + 'gallery/remove' #Removes a gallery and all of its subjects.

gallery_name = "MIT_Dataset"
api_key ='777258c20fecaebf5060918825cf821a'
api_id ='92c17641'

headers = {
  'Content-Type': 'application/json',
  'app_id': api_id,
  'app_key': api_key
}

In [ ]:
def enroll(img_path_in_base64, subject_id, gallery_name=gallery_name):  

  payload = {
    "image": img_path_in_base64, 
    "subject_id": subject_id,
    "gallery_name": gallery_name
}

  response = requests.post(url_enroll, json=payload, headers=headers)
  json_response = response.json()
  if response.status_code != 200 or 'Errors' in json_response:
      raise ServiceRequestError(response.status_code, json_response, payload)

  #print("ENROLL: ", json.dumps(json_response, indent = 4))

In [ ]:
def init():
  global subject_id_list, result_map
  person_already_enrolled = " "
  subject_id_list.clear()
  result_map.clear()
  
  for filename in sorted(os.listdir(dataset_path)):
    #print(filename)
    key = filename.split('.')[0] #get file names without extension
    result_map.update({key : None}) #insert keys in map

    if key.startswith(person_already_enrolled):
      continue

    complete_file_path = dataset_path + key + file_extension
    subject_id = key.split('_')[0]
    subject_id_list.append(subject_id) #insert subject id in list
    print("Enrolling ", subject_id, "...")
    enroll(extract_base64_contents(complete_file_path), subject_id)
    person_already_enrolled = key.split('_')[0]
  print("subject_id_list: ", subject_id_list)
  #print(json.dumps(result_map, indent = 4))

In [ ]:
def list_all_galleries():  
  response = requests.post(url_list_all_galleries, headers=headers)
  json_response = response.json()
  if response.status_code != 200 or 'Errors' in json_response:
      raise ServiceRequestError(response.status_code, json_response, {})

  print("LIST_ALL: ", json.dumps(json_response, indent = 4))

In [ ]:
def view_gallery():
  payload = {'gallery_name': gallery_name}
  response = requests.post(url_view_gallery, json=payload, headers=headers)
  json_response = response.json()
  if response.status_code != 200 or 'Errors' in json_response:
      raise ServiceRequestError(response.status_code, json_response, payload)

  print("VIEW_GALLERY: ", json.dumps(json_response, indent = 4))

In [ ]:
def view_subject(subject_id):
  payload = {
      'gallery_name': gallery_name,
      'subject_id': subject_id
      }
  response = requests.post(url_view_subject, json=payload, headers=headers)
  json_response = response.json()
  if response.status_code != 200 or 'Errors' in json_response:
      raise ServiceRequestError(response.status_code, json_response, payload)

  print("VIEW_SUBJECT: ", json.dumps(json_response, indent = 4))

In [ ]:
def recognize(img_path): 

  payload = {
    'image': img_path,
    'gallery_name': gallery_name,
    'threshold ' : .00 #not required, only for testing purposes
  }

  response = requests.post(url_recognize, json=dict(payload), headers=headers)
  response_json = response.json()

  if response.status_code != 200 or 'Errors' in response_json:
      raise ServiceRequestError(response.status_code, response_json, payload)
  else:
    print("RECOGNIZE: ", json.dumps(response_json, indent = 4))
    candidate_map = dict()

    for candidate in response_json["images"][0]["candidates"]:

      candidate_subject_id = candidate["subject_id"]
      candidate_confidence = float(candidate["confidence"]) * 100 # 0.86944 -> 86.944
      candidate_map.update({candidate_subject_id : candidate_confidence})

    print("RECOGNIZE: ", json.dumps(candidate_map, indent = 4))
    return candidate_map

In [ ]:
def verify(img_path, subject_id): 

  payload = {
    'image': img_path,
    'gallery_name': gallery_name,
    'subject_id' : subject_id,
    'threshold ' : .00 #not required, only for testing purposes
  }

  response = requests.post(url_verify, json=dict(payload), headers=headers)
  response_json = response.json()

  if response.status_code != 200 or 'Errors' in response_json:
      raise ServiceRequestError(response.status_code, response_json, payload)
  else:

    candidate_confidence = round(float(response_json["images"][0]["transaction"]["confidence"]) * 100,2) # 0.86944 -> 86.94
    return candidate_confidence

    #print("verify: ", json.dumps(candidate_map, indent = 4))

In [ ]:
def add_results():
  global subject_id_list, result_map

  for key in result_map.keys():
    
    person_result_list = list()

    for subject_id in subject_id_list:

      complete_file_path = dataset_path + key + file_extension
      confidence = verify(extract_base64_contents(complete_file_path), subject_id)
      print("key:", key, " subject_id:", subject_id, " confidence:", confidence)

      person_result_list.append(confidence)
      
    result_map.update({key : person_result_list})

  #print(json.dumps(result_map, indent = 4))

In [ ]:
def remove_gallery():
  payload = {'gallery_name': gallery_name}
  response = requests.post(url_remove_gallery, json=payload, headers=headers)
  json_response = response.json()
  if response.status_code != 200 or 'Errors' in json_response:
      raise ServiceRequestError(response.status_code, json_response, payload)

  print("REMOVE_ALL: ", json.dumps(json_response, indent = 4))

In [ ]:
def show_results():
  print("========================================FINAL RESULTS========================================")
  for key, value in sorted (result_map.items()):
    print(key, ":", value)

In [ ]:
init()
view_gallery()
view_subject("0")
add_results()
show_results()
remove_gallery() 

key: 0_1  subject_id: 0  confidence: 100.0
key: 0_1  subject_id: 1  confidence: 35.48
key: 0_1  subject_id: 2  confidence: 0.0
key: 0_1  subject_id: 3  confidence: 25.77
key: 0_1  subject_id: 4  confidence: 9.57
key: 0_1  subject_id: 5  confidence: 18.27
key: 0_1  subject_id: 6  confidence: 14.55
key: 0_1  subject_id: 7  confidence: 14.3
key: 0_1  subject_id: 8  confidence: 30.01
key: 0_1  subject_id: 9  confidence: 26.44
key: 0_2  subject_id: 0  confidence: 81.21
key: 0_2  subject_id: 1  confidence: 43.15
key: 0_2  subject_id: 2  confidence: 5.71
key: 0_2  subject_id: 3  confidence: 17.08
key: 0_2  subject_id: 4  confidence: 16.8
key: 0_2  subject_id: 5  confidence: 24.23
key: 0_2  subject_id: 6  confidence: 4.38
key: 0_2  subject_id: 7  confidence: 6.54
key: 0_2  subject_id: 8  confidence: 26.27
key: 0_2  subject_id: 9  confidence: 27.03
key: 0_3  subject_id: 0  confidence: 69.32
key: 0_3  subject_id: 1  confidence: 29.34
key: 0_3  subject_id: 2  confidence: 4.78
key: 0_3  subject_id